# Hyperparameter Finetuning with RandomizedSeachCV
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [1]:
import os
# ggf muss Pfad angepasst werden
os.chdir("{}/..".format(os.getcwd()))
os.getcwd()
import pandas as pd
import numpy as np
from sklearn import model_selection, linear_model, metrics
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, cross_validate
from catboost import CatBoostClassifier, Pool

In [2]:
%run notebooks/utils.ipynb

## Loading Data
Loading the whole training set and adding the column "totalScannedItems" by calling `add_new_features` defined in utils

In [3]:
df = pd.read_csv("data/train.csv", sep="|")
df = add_new_features(df)
sum_frauds, sum_non_frauds  = len(df[df.fraud == 1]), len(df[df.fraud == 0])
train_y = df.fraud
train_x = df.drop(columns=['fraud'])

train_pool = Pool(train_x, train_y)

## Load validation data


In [4]:
# Train Data for validation
df_fit = pd.read_csv("data/train_new.csv", sep="|")
df_fit = add_new_features(df_fit)
df_fit_y = df_fit.fraud
df_fit_x = df_fit.drop(columns=['fraud'])
test_train_pool = Pool(df_fit_x, df_fit_y)

# Validation Data
df_val = pd.read_csv("data/val_new.csv", sep="|")
df_val = add_new_features(df_val)
df_val_y = df_val.fraud
df_val_x = df_val.drop(columns=['fraud'])
test_val_pool = Pool(df_val_x, df_val_y)


## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [5]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172)}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [6]:
params = {
    "iterations": randint(100, 1000),  # default 3
    "learning_rate" :  uniform(1e-6, 1e-2),
    "depth" : randint(1, 10),
    "l2_leaf_reg" : uniform(1e-5,10),
    "border_count" : randint(1,128) #Set it to value of 1-10 might help control the update.

}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [7]:
model = CatBoostClassifier(loss_function="CrossEntropy", task_type="GPU", logging_level="Silent")

## Defining some RandomizedSeach parameters
- cv = cross validation: 3 is the standard value. This is enough and should not be touched
- param_distribution: The params defined two cells above
- scorer: The Scorere defined under "scoring functions" AUC and Fbeta are currently the best.
- return_train_score: Doesnt affect the hyper param search
- refit : Here, you can adress a score with the name, given in the dictionary. 
- n_jobs : -1 to use all cpus
- n_iter : depends on number of params. For 9 params, i suggest a value above 20k. For less paramters 10k could be a good value.

Further informations: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html

In [8]:
search = RandomizedSearchCV(model, scoring=scoring, param_distributions=params, random_state=42, n_iter=10,
                            cv=3, verbose=1, n_jobs=1, return_train_score=True,refit='AUC')
search.fit(train_x, train_y)
results = search.cv_results_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.0min finished
/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [9]:
scorings = {"DMC" : own_scorer, "DMC_Norm" : own_scorer_normalized}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=df_fit, df_val=df_val)

Results Fix Split: 
DMC Score: 90  ---  Normalized DMC Score: 0.2393617021276596, 

Results Cross Validation: 
DMC Score: 34.0  ---  Normalized DMC Score: 0.0903758865248227 


## Show false predictions

In [10]:
res_df = result_dict['dataframe']
res_df[(res_df.prediction != res_df.fraud)]

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScannedItems,prediction,probablity
87,1,1486,57.13,4,1,1,0.015478,0.038445,0.173913,0,23.0,1.0,0.562


## Best Estimator

In [11]:
search.best_estimator_